In [2]:
from scipy.stats import norm, t
import scipy
import numpy as np
import pandas as pd
import os
from datetime import datetime
from functools import partial
from math import log, sqrt, exp, isclose
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings

# problem 1

the closed form greeks for GBSM

In [3]:
def calculate_d1(S, X, T, implied_vol, b):
  return (np.log(S / X) + (b + implied_vol ** 2 / 2) * T) / (implied_vol * np.sqrt(T))

def calculate_d2(d1, T, implied_vol):
  return d1 - implied_vol * np.sqrt(T)

In [4]:
def gbsm_delta(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  delta = norm.cdf(d1 * is_call, 0, 1) * is_call
  return delta

def gbsm_gamma(option_type, S, X, T, implied_vol, r, b):
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  gamma = norm.pdf(d1, 0, 1) / (S * implied_vol * np.sqrt(T))
  return gamma

def gbsm_vega(option_type, S, X, T, implied_vol, r, b):
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  vega = S * norm.pdf(d1, 0, 1) * np.sqrt(T)
  return vega

def gbsm_theta(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  theta = -S * np.exp((b - r) * T) * norm.pdf(d1, 0, 1) * implied_vol / (2 * np.sqrt(T)) \
          -(b - r) * S * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call \
          -r * X * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
  return theta

def gbsm_rho(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  rho = X * T * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
  return rho

def gbsm_carry_rho(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  carry_rho = S * T * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call
  return carry_rho

a finite difference derivative calculation

In [5]:
import inspect

# calculate first order derivative
def first_order_der(func, x, delta):
  return (func(x + delta) - func(x - delta)) / (2 * delta)

# calculate second order derivative
def second_order_der(func, x, delta):
  return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
  # initialize for argument names and order
  arg_names = list(inspect.signature(func).parameters.keys())
  derivative_fs = {1: first_order_der, 2: second_order_der}

  def partial_derivative(*args, **kwargs):
    # parse argument names and order
    args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
    arg_val = args_dict.pop(arg_name)

    def partial_f(x):
      p_kwargs = {arg_name:x, **args_dict}
      return func(**p_kwargs)
    return derivative_fs[order](partial_f, arg_val, delta)
  return partial_derivative

In [6]:
def gbsm(option_type, S, X, T, implied_vol, r, b):
  d1 = (np.log(S / X) + (b + 0.5 * implied_vol ** 2) * T) / (implied_vol * np.sqrt(T))
  d2 = d1 - implied_vol * np.sqrt(T)
  is_call = 1 if option_type == "Call" else -1

  res = is_call * (S * np.e ** ((b - r) * T) * \
                   scipy.stats.norm(0, 1).cdf(is_call * d1) \
                   - X * np.e ** (-r * T) * \
                   scipy.stats.norm(0, 1).cdf(is_call * d2))

  return res

given information

In [7]:
current_date = datetime(2022, 3, 13)
expire_date = datetime(2022, 4, 15)
T = (expire_date - current_date).days / 365

S = 151.03
X = 165
implied_vol = 0.2

r = 0.0425
coupon = 0.0053
b = r - coupon

result

In [16]:
# delta
delta_call = gbsm_delta("Call", S, X, T, implied_vol, r, b)
delta_put = gbsm_delta("Put", S, X, T, implied_vol, r, b)
gbsm_delta_num = cal_partial_derivative(gbsm, 1, 'S')
delta_call_num = gbsm_delta_num("Call", S, X, T, implied_vol, r, b)
delta_put_num = gbsm_delta_num("Put", S, X, T, implied_vol, r, b)
print("delta: (call, put)")
print(f"GBSM:{delta_call}, {delta_put}")
print(f"FDD:{delta_call_num}, {delta_put_num}")

# gamma
gamma_call = gbsm_gamma("Call", S, X, T, implied_vol, r, b)
gamma_put = gbsm_gamma("Put", S, X, T, implied_vol, r, b)
gbsm_gamma_num = cal_partial_derivative(gbsm, 2, 'S')
gamma_call_num = gbsm_gamma_num("Call", S, X, T, implied_vol, r, b)
gamma_put_num = gbsm_gamma_num("Put", S, X, T, implied_vol, r, b)
print("gamma: (call, put)")
print(f"GBSM:{gamma_call}, {gamma_put}")
print(f"FDD:{gamma_call_num}, {gamma_put_num}")

# vega
vega_call = gbsm_vega("Call", S, X, T, implied_vol, r, b)
vega_put = gbsm_vega("Put", S, X, T, implied_vol, r, b)
gbsm_vega_num = cal_partial_derivative(gbsm, 1, 'implied_vol')
vega_call_num = gbsm_vega_num("Call", S, X, T, implied_vol, r, b)
vega_put_num = gbsm_vega_num("Put", S, X, T, implied_vol, r, b)
print("vega: (call, put)")
print(f"GBSM:{vega_call}, {vega_put}")
print(f"FDD:{vega_call_num}, {vega_put_num}")

# theta
theta_call = gbsm_theta("Call", S, X, T, implied_vol, r, b)
theta_put = gbsm_theta("Put", S, X, T, implied_vol, r, b)
gbsm_theta_num = cal_partial_derivative(gbsm, 1, 'T')
theta_call_num = -gbsm_theta_num("Call", S, X, T, implied_vol, r, b)
theta_put_num = -gbsm_theta_num("Put", S, X, T, implied_vol, r, b)
print("theta: (call, put)")
print(f"GBSM:{theta_call}, {theta_put}")
print(f"FDD:{theta_call_num}, {theta_put_num}")

# rho
rho_call = gbsm_rho("Call", S, X, T, implied_vol, r, b)
rho_put = gbsm_rho("Put", S, X, T, implied_vol, r, b)
gbsm_rho_num = cal_partial_derivative(gbsm, 1, 'r')
rho_call_num = gbsm_rho_num("Call", S, X, T, implied_vol, r, b)
rho_put_num = gbsm_rho_num("Put", S, X, T, implied_vol, r, b)
print("rho: (call, put)")
print(f"GBSM:{rho_call}, {rho_put}")
print(f"FDD:{rho_call_num}, {rho_put_num}")

# carry rho
carry_rho_call = gbsm_carry_rho("Call", S, X, T, implied_vol, r, b)
carry_rho_put = gbsm_carry_rho("Put", S, X, T, implied_vol, r, b)
gbsm_carry_rho_num = cal_partial_derivative(gbsm, 1, 'b')
carry_rho_call_num = gbsm_carry_rho_num("Call", S, X, T, implied_vol, r, b)
carry_rho_put_num = gbsm_carry_rho_num("Put", S, X, T, implied_vol, r, b)
print("carry rho: (call, put)")
print(f"GBSM:{carry_rho_call}, {carry_rho_put}")
print(f"FDD:{carry_rho_call_num}, {carry_rho_put_num}")

delta: (call, put)
GBSM:0.08301107089626869, -0.9169889291037313
FDD:0.08297130374668171, -0.9165496329472944
gamma: (call, put)
GBSM:0.016830979206204362, 0.016830979206204362
FDD:0.016822911064195978, 0.016822951920403284
vega: (call, put)
GBSM:6.942036604441163, 6.942036604441163
FDD:6.938653056250743, 6.93865305626673
theta: (call, put)
GBSM:-8.126522359668838, -1.9409914783019566
FDD:-8.126308803761084, -1.9407779203106656
rho: (call, put)
GBSM:1.1025939156368187, -13.758003122735788
FDD:-0.030359909416688424, -1.2427313238703164
carry rho: (call, put)
GBSM:1.132953825011723, -12.515271800549371
FDD:1.1329550097096686, -12.515270634423814


Calculate the binomial tree valuation for American options

In [18]:
def n_nodes(N):
    return (N + 2) * (N + 1) // 2

def node_index(i, j):
    return n_nodes(j - 1) + i

def binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N):
  is_call = 1 if option_type == "Call" else -1
  dt = T / N
  disc = np.exp(-r * dt)
  u = np.exp(implied_vol * np.sqrt(dt))
  d = 1 / u
  p = (np.exp(r * dt) - d) / (u - d)

  C = np.empty(n_nodes(N), dtype=float)

  for i in np.arange(N, -1, -1):
    for j in range(i, -1, -1):
      S = S0 * u ** j * d ** (i - j)
      index = node_index(j, i)
      C[index] = max(0, (S - X) * is_call)
      if i < N:
        val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
        C[index] = max(C[index], val)

  return C[0]

def binomial_tree(option_type, S0, X, T, div_time, div, implied_vol, r, N):
  if div_time is None or div is None:
    return binomial_tree_no_div(option_type, S0, X, T, implied_vol, r, N)

  is_call = 1 if option_type == "Call" else -1
  dt = T / N
  disc = np.exp(-r * dt)

  #calculate u, d, and p
  u = np.exp(implied_vol * np.sqrt(dt))
  d = 1 / u
  p = (np.exp(r * dt) - d) / (u - d)

  new_T = T - div_time * dt
  new_N = N - div_time

  C = np.empty(n_nodes(div_time), dtype=float)
  for i in range(div_time, -1, -1):
    for j in range(i, -1, -1):
      S = S0 * u ** j * d ** (i - j)
      val_exe = max(0, (S - X) * is_call)
      if i < div_time:
        val = disc * (p * C[node_index(j + 1, i + 1)] + (1 - p) * C[node_index(j, i + 1)])
      else:
        val = binomial_tree(option_type, S - div, X, new_T, None, None, implied_vol, r, new_N)
      C[node_index(j, i)] = max(val_exe, val)

  return C[0]

In [19]:
# Assume N is 300
N = 300
value_no_div_call = binomial_tree_no_div("Call", S, X, T, implied_vol, r, N)
value_no_div_put = binomial_tree_no_div("Put", S, X, T, implied_vol, r, N)
print("Binomial tree value without dividend for call: " + str(value_no_div_call))
print("Binomial tree value without dividend for put: " + str(value_no_div_put))

Binomial tree value without dividend for call: 0.341603945513609
Binomial tree value without dividend for put: 14.020014650639181


In [20]:
div_date = datetime(2022, 4, 11)
div = 0.88
div_time = int((div_date - current_date).days / (expire_date - current_date).days * N)

value_call = binomial_tree("Call", S, X, T, div_time, div, implied_vol, r, N)
value_put = binomial_tree("Put", S, X, T, div_time, div, implied_vol, r, N)
print("Binomial tree value with dividend for call: " + str(value_call))
print("Binomial tree value with dividend for put: " + str(value_put))

Binomial tree value with dividend for call: 0.30041508863955924
Binomial tree value with dividend for put: 14.560397204371986


Greeks of American options

In [22]:
# delta
cal_amr_delta_num = cal_partial_derivative(binomial_tree, 1, 'S0')
delta_call_amr = cal_amr_delta_num("Call", S, X, T, div_time, div, implied_vol, r, N)
delta_put_amr = cal_amr_delta_num("Put", S, X, T, div_time, div, implied_vol, r, N)
print("delta: (call, put)")
print(delta_call_amr, delta_put_amr)

# gamma
cal_amr_gamma_num = cal_partial_derivative(binomial_tree, 2, 'S0', delta=1)
gamma_call_amr = cal_amr_gamma_num("Call", S, X, T, div_time, div, implied_vol, r, N)
gamma_put_amr = cal_amr_gamma_num("Put", S, X, T, div_time, div, implied_vol, r, N)
print("gamma: (call, put)")
print(gamma_call_amr, gamma_put_amr)

# vega
cal_amr_vega_num = cal_partial_derivative(binomial_tree, 1, 'implied_vol')
vega_call_amr = cal_amr_vega_num("Call", S, X, T, div_time, div, implied_vol, r, N)
vega_put_amr = cal_amr_vega_num("Put", S, X, T, div_time, div, implied_vol, r, N)
print("vega: (call, put)")
print(vega_call_amr, vega_put_amr)

# theta
cal_amr_theta_num = cal_partial_derivative(binomial_tree, 1, 'T')
theta_call_amr = -cal_amr_theta_num("Call", S, X, T, div_time, div, implied_vol, r, N)
theta_put_amr = -cal_amr_theta_num("Put", S, X, T, div_time, div, implied_vol, r, N)
print("theta: (call, put)")
print(theta_call_amr, theta_put_amr)

# rho
cal_amr_rho_num = cal_partial_derivative(binomial_tree, 1, 'r')
rho_call_amr = cal_amr_rho_num("Call", S, X, T, div_time, div, implied_vol, r, N)
rho_put_amr = cal_amr_rho_num("Put", S, X, T, div_time, div, implied_vol, r, N)
print("rho: (call, put)")
print(rho_call_amr, rho_put_amr)

delta: (call, put)
0.07564529288392463 -0.9304253539674789
gamma: (call, put)
0.01624307464913788 0.014779533669457834
vega: (call, put)
6.42956107974818 6.0562654358991
theta: (call, put)
-7.5594852571106985 -0.8555281205477883
rho: (call, put)
0.9528888093222332 -12.430363528616262


Calculate the sensitivity of the put and call to a change in the dividend amount

In [23]:
delta = 1e-3
call_value1 = binomial_tree("Call", S, X, T, div_time, div + delta, implied_vol, r, N)
call_value2 = binomial_tree("Call", S, X, T, div_time, div - delta, implied_vol, r, N)
call_sens_to_div_amount = (call_value1 - call_value2) / (2*delta)

put_value1 = binomial_tree("Put", S, X, T, div_time, div + delta, implied_vol, r, N)
put_value2 = binomial_tree("Put", S, X, T, div_time, div - delta, implied_vol, r, N)
put_sens_to_div_amount = (put_value1 - put_value2) / (2*delta)
print(f"Sensitivity to dividend amount: Call: {call_sens_to_div_amount:.3f}, Put: {put_sens_to_div_amount:.3f}")

Sensitivity to dividend amount: Call: -0.034, Put: 0.934


# problem 2

In [24]:
def implied_vol_american(option_type, S0, X, T, div_time, div, r, N, market_price, x0=0.5):
  def equation(implied_vol):
    return binomial_tree(option_type, S0, X, T, div_time, div, implied_vol, r, N) - market_price
  # Back solve the binomial tree valuation to get the implied volatility
  return scipy.optimize.fsolve(equation, x0=x0, xtol=0.00001)[0]

def calculate_sim_values(portfolios, sim_prices, days_ahead=0):
  sim_values = pd.DataFrame(index=portfolios.index,
                            columns=list(range(sim_prices.shape[0])))
  sim_prices = np.array(sim_prices)
  for i in portfolios.index:
    if portfolios["Type"][i] == "Stock":
      # For stock, the single value is its price
      single_values = sim_prices
    else:
      # For option, calculate values with gbsm method
      option_type = portfolios["OptionType"][i]
      X = portfolios["Strike"][i]
      T = ((portfolios["ExpirationDate"][i] - current_date).days - days_ahead) / 365
      implied_vol = portfolios["ImpliedVol"][i]
      div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
      div = 1
      option_values = []
      for S in sim_prices:
        option_values.append(binomial_tree(option_type, S, X, T, div_time, div, implied_vol, r, N))
      single_values = np.array(option_values)

    # Calculate the total values based on holding
    sim_values.loc[i, :] = portfolios["Holding"][i] * single_values

  # Combine the values for same portfolios
  sim_values['Portfolio'] = portfolios['Portfolio']
  return sim_values.groupby('Portfolio').sum()

Fit a Normal distribution and calculate Mean, VaR and ES.

In [30]:
portfolios = pd.read_csv('problem2.csv', parse_dates=['ExpirationDate'])
portfolios['CurrentValue'] = portfolios['CurrentPrice'] * portfolios['Holding']

S = 165
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

# Calculate the implied volatility for all portfolios
implied_vols = []
for i in range(len(portfolios.index)):
  if portfolios["Type"][i] == "Stock":
    implied_vols.append(None)
  else:
    option_type = portfolios["OptionType"][i]
    X = portfolios["Strike"][i]
    T = (portfolios["ExpirationDate"][i] - current_date).days / 365
    div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
    market_price = portfolios["CurrentPrice"][i]
    implied_vol = implied_vol_american(option_type, S, X, T, div_time, div, r, N, market_price)
    implied_vols.append(implied_vol)

# Store the implied volatility in portfolios
portfolios["ImpliedVol"] = implied_vols

<ipython-input-18-b31624876635>:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  C[index] = max(0, (S - X) * is_call)
<ipython-input-18-b31624876635>:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  C[index] = max(C[index], val)
<ipython-input-18-b31624876635>:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  C[node_index(j, i)] = max(val_exe, val)
<ipython-input-24-7e60cce3edde>:5: RuntimeWarning: The iteration is not making good progress, as measured by the 
  

In [31]:
def return_calculate(prices: pd.DataFrame, method: str = "DISCRETE", dateColumn: str = "Date"):

    vars = list(prices.columns)
    nVars = len(vars)
    vars.remove(dateColumn)
    if nVars == len(vars):
        raise ValueError("dateColumn: " + dateColumn + " not in DataFrame: " + str(vars))
    nVars = nVars-1

    p = np.array(prices[vars])
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.empty((n-1,m))

    for i in range(n-1):
        for j in range(m):
            p2[i,j] = p[i+1,j] / p[i,j]

    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError("method: " + method + " must be in (\"LOG\",\"DISCRETE\")")

    dates = prices[dateColumn][1:n]
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars[i]] = p2[:,i]
    return out

In [32]:
all_prices = pd.read_csv("DailyPrices.csv")
all_returns = return_calculate(all_prices, method= "LOG", dateColumn="Date")

<ipython-input-31-6951b6cad39e>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [33]:
# Simulate the prices based on returns with normal distribution
std = all_returns['AAPL'].std()
np.random.seed(0)
sim_returns = scipy.stats.norm(0, std).rvs((10, 100))
sim_prices = 151.03 * (1 + sim_returns).prod(axis=0)

# Calculate the current values and sim values
portfolios["CurrentValue"] = portfolios["CurrentPrice"] * portfolios["Holding"]
curr_values = portfolios.groupby('Portfolio')['CurrentValue'].sum()
sim_values = calculate_sim_values(portfolios, sim_prices, 10)

# Calculate the value difference
sim_value_changes = (sim_values.T - curr_values).T

In [34]:
def cal_VaR_ES_hist(returns, alpha=0.05):
    returns = sorted(returns)
    n = alpha * len(returns)
    iup = int(np.ceil(n))
    idn = int(np.floor(n))
    VaR = (returns[iup] + returns[idn]) / 2

    ES = np.mean(returns[0:idn])

    return -VaR, -ES, returns

In [35]:
# Calculate the Mean, VaR and ES, and print the results
resulting_mat = pd.DataFrame(0, index=sim_values.index.values, columns=["Mean", "VaR($)", "ES($)"])
for i in range(len(resulting_mat)):
    resulting_mat.iloc[i,0] = sim_values.iloc[i,:].mean()
    resulting_mat.iloc[i,1] = cal_VaR_ES_hist(sim_value_changes.iloc[i,:],alpha=0.05)[0]
    resulting_mat.iloc[i,2] = cal_VaR_ES_hist(sim_value_changes.iloc[i,:],alpha=0.05)[1]

<ipython-input-35-2e3cacb6b691>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.09736091931446' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resulting_mat.iloc[i,0] = sim_values.iloc[i,:].mean()
<ipython-input-35-2e3cacb6b691>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.7990173992004115' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resulting_mat.iloc[i,1] = cal_VaR_ES_hist(sim_value_changes.iloc[i,:],alpha=0.05)[0]
<ipython-input-35-2e3cacb6b691>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.799952956514669' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  resulting_mat.iloc[i,2] = cal_VaR_ES_hist(sim

In [36]:
resulting_mat

,Mean,VaR($),ES($)
Call,3.097361,6.799017,6.799953
CallSpread,2.659462,4.589017,4.589953
CoveredCall,149.117511,7.260838,10.602005
ProtectedPut,156.756094,3.003057,4.272518
Put,9.631896,-0.268636,0.799042
PutSpread,5.011379,-0.219114,0.373946
Stock,150.074736,11.310838,14.652005
Straddle,12.729257,0.973351,0.984207
SynLong,-6.534535,17.095658,19.424308


Calculate VaR and ES using Delta-Normal


In [37]:
cal_amr_delta_num =  cal_partial_derivative(binomial_tree, 1, 'S0')

In [38]:
# Calculate the implied volatility for all portfolios
deltas = []
for i in range(len(portfolios.index)):
  if portfolios["Type"][i] == "Stock":
    deltas.append(1)
  else:
    option_type = portfolios["OptionType"][i]
    X = portfolios["Strike"][i]
    T = ((portfolios["ExpirationDate"][i] - current_date).days - 10) / 365
    div_time = int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)
    delta = cal_amr_delta_num(option_type, S, X, T, div_time, div, implied_vol, r, N)
    deltas.append(delta)

In [49]:
# Store the deltas in portfolios
portfolios["deltas"] = deltas

alpha = 0.05
t = 10
result_dn = pd.DataFrame(index=sorted(portfolios['Portfolio'].unique()), columns=['Mean', 'VaR', 'ES'])
result_dn.index.name = 'Portfolio'
for pfl, df in portfolios.groupby('Portfolio'):
  gradient = S / df['CurrentValue'].sum() * (df['Holding'] * df['deltas']).sum()
  pfl_10d_std = abs(gradient) * std * np.sqrt(t)
  N = scipy.stats.norm(0, 1)
  present_value = df['CurrentValue'].sum()
  result_dn.loc[pfl]['Mean'] = 0
  result_dn.loc[pfl]['VaR'] = -present_value * N.ppf(alpha) * pfl_10d_std
  result_dn.loc[pfl]['ES'] = present_value * pfl_10d_std * N.pdf(N.ppf(alpha)) / alpha

print(result_dn)

             Mean        VaR         ES
Portfolio                              
Call            0   9.922584  12.443321
CallSpread      0    3.36575   4.220786
CoveredCall     0   3.717425   4.661801
ProtectedPut    0   9.887767  12.399659
Put             0   2.265449   2.840964
PutSpread       0   1.102982   1.383185
Stock           0  12.108229  15.184207
Straddle        0   7.657136   9.602357
SynLong         0  12.188033  15.284285


<ipython-input-49-5c678d7e05d0>:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  result_dn.loc[pfl]['Mean'] = 0
<ipython-input-49-5c678d7e05d0>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting 

Rerun the code from last week using inputs in this week project

In [66]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import root_scalar
from statsmodels.tsa.arima.model import ARIMA

# Inputs
currentS = 165
current_dt = datetime(2023, 3, 3)
rf = 0.0425  # Risk-Free Rate
dy = 0.0057  # Dividend Yield
nSim = 10000
fwdT = 10

# Load Data
returns = pd.read_csv("DailyPrices.csv", parse_dates=["Date"])
returns["returns"] = np.log(returns["AAPL"]).diff().dropna() - np.mean(np.log(returns["AAPL"]).diff().dropna())
portfolio = pd.read_csv("problem2.csv")

# Convert ExpirationDate for Options to datetime
portfolio["ExpirationDate"] = [
    pd.to_datetime(portfolio["ExpirationDate"][i], format="%m/%d/%Y")
    if portfolio["Type"][i] == "Option" else None
    for i in range(len(portfolio))
]

# Define GBSM (Black-Scholes-Merton)
def gbsm(is_call, S, K, T, r, q, sigma):
    from scipy.stats import norm
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if is_call:
        return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

# Calculate implied volatility
def implied_vol(option_type, S, K, T, r, q, current_price):
    def objective(vol):
        return gbsm(option_type == "Call", S, K, T, r, q, vol) - current_price

    try:
        result = root_scalar(objective, bracket=[1e-5, 10], method='bisect') # Wider bracket
        return result.root if result.converged else np.nan  # Return NaN if not converged
    except ValueError:
        print(f"Warning: Could not find implied volatility for option with parameters: "
              f"option_type={option_type}, S={S}, K={K}, T={T}, r={r}, q={q}, current_price={current_price}")
        return np.nan  # Return NaN in case of ValueError

portfolio["ImpVol"] = [
    implied_vol(portfolio["OptionType"][i], currentS, portfolio["Strike"][i],
                (portfolio["ExpirationDate"][i] - current_dt).days / 365, 0.05, 0.02,
                portfolio["CurrentPrice"][i]) if portfolio["Type"][i] == "Option" else None
    for i in range(len(portfolio))
]

# Fit AR(1) model
model = ARIMA(returns["returns"], order=(1, 0, 0)).fit()
mean = model.params["const"]
phi = model.params["ar.L1"]
sigma = np.sqrt(model.resid.var())

def ar1_simulation(y, mean, phi, sigma, innovations, ahead=1):
    l = len(y)
    n = len(innovations) // ahead
    out = np.zeros((ahead, n))
    y_last = y.iloc[-1] - mean
    for i in range(n):
        yl = y_last
        for j in range(ahead):
            next_val = phi * yl + sigma * innovations[i * ahead + j]
            yl = next_val
            out[j, i] = next_val
    return out + mean

# Simulate AR(1) paths forward
innovations = np.random.randn(fwdT * nSim)
arSim = ar1_simulation(returns["returns"], mean, phi, sigma, innovations, ahead=fwdT)

# Calculate simulated returns and prices
simReturns = np.sum(arSim, axis=0)
simPrices = currentS * np.exp(simReturns)

# Expand portfolio for simulations
iteration = np.arange(1, nSim + 1)
values = portfolio.merge(pd.DataFrame({"iteration": iteration}), how="cross")
nVals = len(values)

# Set forward time-to-maturity
values["fwd_ttm"] = [
    (values["ExpirationDate"][i] - current_dt - timedelta(days=fwdT)).days / 365
    if values["Type"][i] == "Option" else None
    for i in range(nVals)
]

# Calculate simulated values and P&L
simulatedValue = []
currentValue = []
pnl = []

for i in range(nVals):
    simprice = simPrices[values["iteration"].iloc[i] - 1]
    current_val = values["Holding"].iloc[i] * values["CurrentPrice"].iloc[i]
    currentValue.append(current_val)

    if values["Type"].iloc[i] == "Option":
        sim_val = values["Holding"].iloc[i] * gbsm(
            values["OptionType"].iloc[i] == "Call",
            simprice, values["Strike"].iloc[i], values["fwd_ttm"].iloc[i],
            rf, rf - dy, values["ImpVol"].iloc[i]
        )
    elif values["Type"].iloc[i] == "Stock":
        sim_val = values["Holding"].iloc[i] * simprice
    else:
        sim_val = 0

    simulatedValue.append(sim_val)
    pnl.append(sim_val - current_val)

values["simulatedValue"] = simulatedValue
values["pnl"] = pnl
values["currentValue"] = currentValue

# Calculate Risk Metrics (Mean, VaR, ES)
portfolio_agg = values.groupby("Portfolio")[["simulatedValue", "currentValue"]].agg(list).reset_index()
portfolio_metrics = []

for idx, row in portfolio_agg.iterrows():
    simulated_values = np.array(row["simulatedValue"])
    current_value = np.sum(row["currentValue"])

    # Mean value
    mean_value = np.mean(simulated_values)

    # VaR (5th percentile of losses)
    losses = current_value - simulated_values
    var_95 = np.percentile(losses, 5)

    # Expected Shortfall (average loss beyond VaR)
    es_95 = losses[losses >= var_95].mean()

    # Append results
    portfolio_metrics.append({
        "Portfolio": row["Portfolio"],
        "Current Value ($)": current_value,
        "Mean Simulated Value ($)": mean_value,
        "VaR 95% ($ Loss)": var_95,
        "ES 95% ($ Loss)": es_95
    })

# Convert to DataFrame for presentation
portfolio_metrics_df = pd.DataFrame(portfolio_metrics)

# Present results
print(portfolio_metrics_df)


      Portfolio  Current Value ($)  Mean Simulated Value ($)  \
0         Call             68000.0                       NaN   
1    CallSpread            45900.0                       NaN   
2   CoveredCall          1469800.0                       NaN   
3  ProtectedPut          1540400.0                 83.861954   
4          Put             48500.0                  4.371051   
5     PutSpread            30100.0                  1.394586   
6         Stock          1510300.0                165.063209   
7      Straddle           116500.0                       NaN   
8       SynLong            19500.0                       NaN   

   VaR 95% ($ Loss)  ES 95% ($ Loss)  
0               NaN              NaN  
1               NaN              NaN  
2               NaN              NaN  
3      1.540225e+06     1.540321e+06  
4      4.849190e+04     4.849590e+04  
5      3.009294e+04     3.009898e+04  
6      1.510122e+06     1.510136e+06  
7               NaN              NaN  
8       

<ipython-input-66-1eb79d08de7e>:140: RuntimeWarning: Mean of empty slice.
  es_95 = losses[losses >= var_95].mean()
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## problem3

In [40]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

# Step 1: Import Data
momentum_factor = pd.read_csv('F-F_Momentum_Factor_daily.CSV', skipfooter=1)
fama_french_factors = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skipfooter=1)
stock_prices = pd.read_csv('DailyPrices.csv')

# Step 2: Data Preprocessing
# Convert date columns to datetime format
momentum_factor['Date'] = pd.to_datetime(momentum_factor['Date'], format='%Y%m%d')
fama_french_factors['Date'] = pd.to_datetime(fama_french_factors['Date'], format='%Y%m%d')
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])



<ipython-input-40-6c3689d0e210>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  momentum_factor = pd.read_csv('F-F_Momentum_Factor_daily.CSV', skipfooter=1)
<ipython-input-40-6c3689d0e210>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  fama_french_factors = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skipfooter=1)


In [41]:
print(momentum_factor.columns)

Index(['Date', 'Mom   '], dtype='object')


In [42]:
# Convert factor values from percentages to decimals
momentum_factor['Mom'] = momentum_factor['Mom   '] / 100
fama_french_factors[['Mkt-RF', 'SMB', 'HML', 'RF']] = fama_french_factors[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100

# Merge Fama-French and momentum factors on the date
factors = pd.merge(fama_french_factors, momentum_factor, on='Date', how='inner')

# Filter data for the past 10 years
end_date = stock_prices['Date'].max()  # Latest date in stock prices
start_date = end_date - pd.DateOffset(years=10)  # 10 years prior
factors = factors[(factors['Date'] >= start_date) & (factors['Date'] <= end_date)]
stock_prices = stock_prices[(stock_prices['Date'] >= start_date) & (stock_prices['Date'] <= end_date)]

# Set 'Date' as the index for stock prices and calculate daily log returns
stock_prices.set_index('Date', inplace=True)
returns = stock_prices.pct_change().dropna()  # Percentage change to get daily returns

# Align daily stock returns with factor data by merging on dates
data = pd.merge(returns, factors, left_index=True, right_on='Date')


In [50]:
print(stock_prices.head())

        Date         SPY        AAPL        MSFT        AMZN        TSLA  \
0 2023-09-05  443.042969  188.734238  331.065002  137.270004  256.489990   
1 2023-09-06  440.064606  181.978806  330.400024  135.360001  251.919998   
2 2023-09-07  438.713501  176.656036  327.452148  137.850006  251.490005   
3 2023-09-08  439.374268  177.272873  331.779633  138.229996  248.500000   
4 2023-09-11  442.263885  178.446869  335.422302  143.100006  273.579987   

        GOOGL        GOOG        META       NVDA  ...         PNC       MDLZ  \
0  135.434647  136.375107  299.534485  48.529320  ...  115.410866  67.877701   
1  134.127884  135.038361  298.556488  47.046768  ...  113.370651  68.004555   
2  134.925903  135.866348  298.057526  46.227013  ...  110.889824  68.307068   
3  136.043137  136.863876  297.279114  45.558216  ...  112.000938  68.131409   
4  136.581802  137.402573  306.929291  45.164337  ...  113.044975  69.643929   

          MO         ADI       GILD         LMT         SYK   

In [44]:
# Step 3: Select Stocks
# List of selected stocks based on the table
selected_stocks = [
    "AAPL", "META", "UNH", "MA", "MSFT", "NVDA", "HD", "PFE",
    "AMZN", "BRK-B", "PG", "XOM", "TSLA", "JPM", "V", "DIS",
    "GOOGL", "JNJ", "BAC", "CSCO"
]

# Reset the index to access the 'Date' column
stock_prices = stock_prices.reset_index()

# Filter stock prices to include only the selected stocks
filtered_stock_prices = stock_prices[['Date'] + selected_stocks].set_index('Date')

# Recalculate daily log returns for the filtered stocks
filtered_returns = filtered_stock_prices.pct_change().dropna()

# Align daily returns with factor data by merging on dates
filtered_data = pd.merge(filtered_returns, factors, left_index=True, right_on='Date')

In [45]:
# Step 4: Fit the 4-Factor Model
# Extract risk-free rate and factor columns for regression
risk_free = filtered_data['RF'].values
X = filtered_data[['Mkt-RF', 'SMB', 'HML', 'Mom']].values

# Initialize dictionaries to store regression coefficients (betas) and expected returns
betas = {}
expected_returns = {}

for stock in filtered_returns.columns:
    # Dependent variable: stock excess returns (daily return - risk-free rate)
    y = filtered_data[stock].values - risk_free

    # Fit the 4-factor model using linear regression
    model = LinearRegression().fit(X, y)
    betas[stock] = model.coef_  # Store factor loadings (betas)

    # Compute expected return using the mean of the factors
    factor_means = factors[['Mkt-RF', 'SMB', 'HML', 'Mom']].mean().values
    expected_returns[stock] = risk_free.mean() + np.dot(model.coef_, factor_means)

# Step 5: Compute the Annualized Covariance Matrix
# Daily covariance matrix of stock returns
daily_cov = filtered_returns.cov()

# Annualize the covariance matrix (252 trading days per year)
annual_cov = daily_cov * 252

In [51]:
print(annual_cov)

           AAPL      META       UNH        MA      MSFT      NVDA        HD  \
AAPL   0.050501  0.020870 -0.001577  0.010380  0.021561  0.033939  0.010867   
META   0.020870  0.136272 -0.011660  0.015539  0.041722  0.077580  0.010079   
UNH   -0.001577 -0.011660  0.049432  0.005066 -0.001927 -0.016474  0.006551   
MA     0.010380  0.015539  0.005066  0.027007  0.012275  0.021657  0.011621   
MSFT   0.021561  0.041722 -0.001927  0.012275  0.039604  0.045418  0.011498   
NVDA   0.033939  0.077580 -0.016474  0.021657  0.045418  0.253770  0.023843   
HD     0.010867  0.010079  0.006551  0.011621  0.011498  0.023843  0.042762   
PFE    0.002864  0.000547  0.010636  0.003087  0.005221 -0.016984  0.005594   
AMZN   0.024202  0.064354 -0.006390  0.015380  0.035189  0.067779  0.016908   
BRK-B  0.006499  0.008382  0.006006  0.010321  0.006653  0.003171  0.010077   
PG    -0.000199 -0.000426  0.007212  0.004023  0.002121 -0.009521  0.003260   
XOM   -0.002407 -0.005812  0.001574  0.004131 -0.005

In [47]:
# Step 6: Portfolio Optimization to Maximize Sharpe Ratio
# Convert expected returns dictionary to a vector
expected_returns_vector = np.array([expected_returns[stock] for stock in filtered_returns.columns])

# Define the Sharpe ratio function to maximize
def sharpe_ratio(weights):
    # Portfolio return
    portfolio_return = np.dot(weights, expected_returns_vector)
    # Portfolio volatility
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))
    # Negative Sharpe ratio for minimization
    return -((portfolio_return - 0.05) / portfolio_volatility)

# Constraints: weights must sum to 1, and no short-selling (weights >= 0)
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = [(0, 1) for _ in range(len(filtered_returns.columns))]

# Initial guess for weights: equal allocation
init_weights = np.ones(len(filtered_returns.columns)) / len(filtered_returns.columns)

# Perform optimization to maximize Sharpe ratio
result = minimize(sharpe_ratio, init_weights, bounds=bounds, constraints=constraints)

# Extract optimal weights
optimal_weights = result.x

# Step 7: Output Results
# Combine stock names with their weights, converted to percentages and rounded to two decimal places
stock_weights = pd.DataFrame({
    "Stock": filtered_returns.columns,
    "Weight (%)": np.round(optimal_weights * 100, 2)
})

# Print results
print(stock_weights.to_string(index=False))

Stock  Weight (%)
 AAPL         0.0
 META         0.0
  UNH         0.0
   MA         0.0
 MSFT         0.0
 NVDA         0.0
   HD         0.0
  PFE         0.0
 AMZN         0.0
BRK-B         0.0
   PG         0.0
  XOM         0.0
 TSLA       100.0
  JPM         0.0
    V         0.0
  DIS         0.0
GOOGL         0.0
  JNJ         0.0
  BAC         0.0
 CSCO         0.0


Modified version：

In [48]:
# Print expected returns for inspection
print("Expected Returns for Each Stock (Annualized):")
for stock, ret in expected_returns.items():
    print(f"{stock}: {ret:.6f}")

# Print the annualized covariance matrix
print("\nAnnualized Covariance Matrix:")
print(annual_cov)

# Define Sharpe ratio with diversification penalty (optional)
def sharpe_ratio(weights):
    portfolio_return = np.dot(weights, expected_returns_vector)  # Portfolio return
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))  # Portfolio risk
    diversification_penalty = 0.001 * np.sum(weights**2)  # Regularization for diversification
    return -((portfolio_return - 0.05) / portfolio_volatility - diversification_penalty)

# Enforce bounds to prevent concentration (optional)
bounds = [(0, 0.3) for _ in range(len(filtered_returns.columns))]  # Max 30% per stock

# Optimize portfolio weights
result = minimize(sharpe_ratio, init_weights, bounds=bounds, constraints=constraints)

# Check optimization result
if not result.success:
    print("Optimization failed:", result.message)
else:
    # Extract optimal weights
    optimal_weights = result.x

    # Output final weights
    stock_weights = pd.DataFrame({
        "Stock": filtered_returns.columns,
        "Weight (%)": np.round(optimal_weights * 100, 2)
    })
    print("\nOptimal Portfolio Weights:")
    print(stock_weights.to_string(index=False))


Expected Returns for Each Stock (Annualized):
AAPL: 0.000705
META: 0.001032
UNH: 0.000235
MA: 0.000591
MSFT: 0.000779
NVDA: 0.001424
HD: 0.000597
PFE: 0.000334
AMZN: 0.000963
BRK-B: 0.000483
PG: 0.000342
XOM: 0.000311
TSLA: 0.001256
JPM: 0.000525
V: 0.000568
DIS: 0.000513
GOOGL: 0.000861
JNJ: 0.000295
BAC: 0.000620
CSCO: 0.000535

Annualized Covariance Matrix:
           AAPL      META       UNH        MA      MSFT      NVDA        HD  \
AAPL   0.050501  0.020870 -0.001577  0.010380  0.021561  0.033939  0.010867   
META   0.020870  0.136272 -0.011660  0.015539  0.041722  0.077580  0.010079   
UNH   -0.001577 -0.011660  0.049432  0.005066 -0.001927 -0.016474  0.006551   
MA     0.010380  0.015539  0.005066  0.027007  0.012275  0.021657  0.011621   
MSFT   0.021561  0.041722 -0.001927  0.012275  0.039604  0.045418  0.011498   
NVDA   0.033939  0.077580 -0.016474  0.021657  0.045418  0.253770  0.023843   
HD     0.010867  0.010079  0.006551  0.011621  0.011498  0.023843  0.042762   
PFE  